In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [ ]:
import fastai
from fastai import*
from fastai.text import*
import pandas as pd
import numpy as np
from functools import partial
import io
import os
import matplotlib.pyplot as plt


#Downloading the data
from sklearn.datasets import fetch_20newsgroups
dataset= fetch_20newsgroups(shuffle=True, random_state=1, remove =('headers','footers','quotes'))
documents=dataset.data

df=pd.DataFrame({'label':dataset.target, 'text':dataset.data})
df.shape

#Binary Classification
df= df[df['label'].isin([1,10])]
df=df.reset_index(drop=True)
df['label'].value_counts()

#Data Preprocessing
# getting rid of everything other tha alphabets
df['text']=df['text'].str.replace("[^a-zA-Z]"," ")

#Eliminating Stopwords(a,an,the,etc)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#Tokenization
tokenized_doc = df['text'].apply(lambda x:x.split())

#remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

#detokenization
detokenized_doc=[]
for i in range(len(df)):
  t=' '.join(tokenized_doc[i])
  detokenized_doc.append(t)

df['text']= detokenized_doc

#splitting up data
from sklearn.model_selection import train_test_split
df_train,df_validation = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state=12)
df_train.shape, df_validation.shape # returns tuple indicating no. of instances in training and testing data
 
#Language Model data
data_lm= TextLMDataBunch.from_df(train_df=df_train, valid_df=df_validation, path="")

#CLassifier Model Data
data_clas= TextClasDataBunch.from_df(path = "", train_df=df_train, valid_df= df_validation, vocab=data_lm.train_ds.vocab, bs=32)

#FIne tuning
learn= language_model_learner(data_lm,arch=AWD_LSTM, pretrained = True , drop_mult=0.7)

#taining the object- learning rate=1e-2
learn.fit_one_cycle(1, 1e-2)

#Save this Encoder
learn.save_encoder('ft_enc')

#Building a Classifier with fine-tuned encoder
learn = text_classifier_learner(data_clas, arch= AWD_LSTM ,drop_mult=0.7)
learn.load_encoder('ft_enc')

#Re-try to fit our model
learn.fit_one_cycle(1,1e-2)

#Predictions
pred, target= learn.get_preds()
prediction=np.argmax(pred, axis=1 )
pd.crosstab(prediction, target, rownames=['predictions'], colnames=['target'])

#plt.plot(pred, target, linestyle='dashed' )
#plt.show()










[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


epoch,train_loss,valid_loss,accuracy,time
0,6.128574,5.214192,0.246747,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.409979,0.292745,0.919831,00:12


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


target,0,1
predictions,,
0,233,37
1,1,203
